In [1]:
import gc  # Garbage collection for memory management
import os  # Operating system-related functions
import time  # Time-related functions
import warnings  # Handling warnings
from itertools import combinations  # For creating combinations of elements
from warnings import simplefilter  # Simplifying warning handling

# 📦 Importing machine learning libraries
import joblib  # For saving and loading models
import lightgbm as lgb  # LightGBM gradient boosting framework
import numpy as np  # Numerical operations
import pandas as pd  # Data manipulation and analysis
from sklearn.metrics import mean_absolute_error  # Metric for evaluation
from sklearn.model_selection import KFold, TimeSeriesSplit  # Cross-validation techniques

# 🤐 Disable warnings to keep the code clean
warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

# 📊 Define flags and variables
is_offline = True  # Flag for online/offline mode
is_train = True  # Flag for training mode
is_infer = True  # Flag for inference mode
max_lookback = np.nan  # Maximum lookback (not specified)
split_day = 477  # Split day for time series data


## Preprocess

In [2]:
def weighted_average(a):
    w = []
    n = len(a)
    for j in range(1, n + 1):
        j = 2 if j == 1 else j
        w.append(1 / (2**(n + 1 - j)))
    return w

In [3]:
# 📂 Read the dataset from a CSV file using Pandas
df = pd.read_csv("optiver-trading-at-the-close/train.csv")

# 🧹 Remove rows with missing values in the "target" column
df = df.dropna(subset=["target"])

# 🔁 Reset the index of the DataFrame and apply the changes in place
df.reset_index(drop=True, inplace=True)

# 📏 Get the shape of the DataFrame (number of rows and columns)
df_shape = df.shape


In [4]:
# 🧹 Function to reduce memory usage of a Pandas DataFrame
def reduce_mem_usage(df, verbose=0):
    """
    Iterate through all numeric columns of a dataframe and modify the data type
    to reduce memory usage.
    """
    
    # 📏 Calculate the initial memory usage of the DataFrame
    start_mem = df.memory_usage().sum() / 1024**2

    # 🔄 Iterate through each column in the DataFrame
    for col in df.columns:
        col_type = df[col].dtype

        # Check if the column's data type is not 'object' (i.e., numeric)
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            
            # Check if the column's data type is an integer
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                # Check if the column's data type is a float
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)

    # ℹ️ Provide memory optimization information if 'verbose' is True
    if verbose:
        logger.info(f"Memory usage of dataframe is {start_mem:.2f} MB")
        end_mem = df.memory_usage().sum() / 1024**2
        logger.info(f"Memory usage after optimization is: {end_mem:.2f} MB")
        decrease = 100 * (start_mem - end_mem) / start_mem
        logger.info(f"Decreased by {decrease:.2f}%")

    # 🔄 Return the DataFrame with optimized memory usage
    return df


In [5]:
# 🏎️ Import Numba for just-in-time (JIT) compilation and parallel processing
from numba import njit, prange

# 📊 Function to compute triplet imbalance in parallel using Numba
@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))

    # 🔁 Loop through all combinations of triplets
    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        
        # 🔁 Loop through rows of the DataFrame
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            
            # 🚫 Prevent division by zero
            if mid_val == min_val:
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)

    return imbalance_features

# 📈 Function to calculate triplet imbalance for given price data and a DataFrame
def calculate_triplet_imbalance_numba(price, df):
    # Convert DataFrame to numpy array for Numba compatibility
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]

    # Calculate the triplet imbalance using the Numba-optimized function
    features_array = compute_triplet_imbalance(df_values, comb_indices)

    # Create a DataFrame from the results
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)

    return features


In [6]:
# 📊 Function to generate imbalance features
def imbalance_features(df):
    # Define lists of price and size-related column names
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]

    # V1 features
    # Calculate various features using Pandas eval function
    df["volume"] = df.eval("ask_size + bid_size")
    df["mid_price"] = df.eval("(ask_price + bid_price) / 2")
    df["liquidity_imbalance"] = df.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    df["matched_imbalance"] = df.eval("(imbalance_size-matched_size)/(matched_size+imbalance_size)")
    df["size_imbalance"] = df.eval("bid_size / ask_size")
    
    # Create features for pairwise price imbalances
    for c in combinations(prices, 2):
        df[f"{c[0]}_{c[1]}_imb"] = df.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]})")

    # Calculate triplet imbalance features using the Numba-optimized function
    for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
        triplet_feature = calculate_triplet_imbalance_numba(c, df)
        df[triplet_feature.columns] = triplet_feature.values
        
    # V2 features
    # Calculate additional features
    df["imbalance_momentum"] = df.groupby(['stock_id'])['imbalance_size'].diff(periods=1) / df['matched_size']
    df["price_spread"] = df["ask_price"] - df["bid_price"]
    df["spread_intensity"] = df.groupby(['stock_id'])['price_spread'].diff()
    df['price_pressure'] = df['imbalance_size'] * (df['ask_price'] - df['bid_price'])
    df['market_urgency'] = df['price_spread'] * df['liquidity_imbalance']
    df['depth_pressure'] = (df['ask_size'] - df['bid_size']) * (df['far_price'] - df['near_price'])
    
    # Calculate various statistical aggregation features
    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)
        
    # V3 features
    # Calculate shifted and return features for specific columns
    for col in ['matched_size', 'imbalance_size', 'reference_price', 'imbalance_buy_sell_flag']:
        for window in [1, 2, 3, 10]:
            df[f"{col}_shift_{window}"] = df.groupby('stock_id')[col].shift(window)
            df[f"{col}_ret_{window}"] = df.groupby('stock_id')[col].pct_change(window)
    
    # Calculate diff features for specific columns
    for col in ['ask_price', 'bid_price', 'ask_size', 'bid_size']:
        for window in [1, 2, 3, 10]:
            df[f"{col}_diff_{window}"] = df.groupby("stock_id")[col].diff(window)

    # Replace infinite values with 0
    return df.replace([np.inf, -np.inf], 0)

# 📅 Function to generate time and stock-related features
def other_features(df):
    df["dow"] = df["date_id"] % 5  # Day of the week
    df["seconds"] = df["seconds_in_bucket"] % 60  # Seconds
    df["minute"] = df["seconds_in_bucket"] // 60  # Minutes

    # Map global features to the DataFrame
    for key, value in global_stock_id_feats.items():
        df[f"global_{key}"] = df["stock_id"].map(value.to_dict())

    return df

# 🚀 Function to generate all features by combining imbalance and other features
def generate_all_features(df):
    # Select relevant columns for feature generation
    cols = [c for c in df.columns if c not in ["row_id", "time_id", "target"]]
    df = df[cols]
    
    # Generate imbalance features
    df = imbalance_features(df)
    
    # Generate time and stock-related features
    df = other_features(df)
    gc.collect()  # Perform garbage collection to free up memory
    
    # Select and return the generated features
    feature_name = [i for i in df.columns if i not in ["row_id", "target", "time_id", "date_id"]]
    
    return df[feature_name]


In [7]:
# Check if the code is running in offline or online mode
if is_offline:
    # In offline mode, split the data into training and validation sets based on the split_day
    df_train = df[df["date_id"] <= split_day]
    df_valid = df[df["date_id"] > split_day]
    
    # Display a message indicating offline mode and the shapes of the training and validation sets
    print("Offline mode")
    print(f"train : {df_train.shape}, valid : {df_valid.shape}")
else:
    # In online mode, use the entire dataset for training
    df_train = df
    
    # Display a message indicating online mode
    print("Online mode")


Offline mode
train : (5204892, 17), valid : (33000, 17)


In [8]:
if is_train:
    global_stock_id_feats = {
        "median_size": df_train.groupby("stock_id")["bid_size"].median() + df_train.groupby("stock_id")["ask_size"].median(),
        "std_size": df_train.groupby("stock_id")["bid_size"].std() + df_train.groupby("stock_id")["ask_size"].std(),
        "ptp_size": df_train.groupby("stock_id")["bid_size"].max() - df_train.groupby("stock_id")["bid_size"].min(),
        "median_price": df_train.groupby("stock_id")["bid_price"].median() + df_train.groupby("stock_id")["ask_price"].median(),
        "std_price": df_train.groupby("stock_id")["bid_price"].std() + df_train.groupby("stock_id")["ask_price"].std(),
        "ptp_price": df_train.groupby("stock_id")["bid_price"].max() - df_train.groupby("stock_id")["ask_price"].min(),
    }
    if is_offline:
        df_train_feats = generate_all_features(df_train)
        print("Build Train Feats Finished.")
        df_valid_feats = generate_all_features(df_valid)
        print("Build Valid Feats Finished.")
        df_valid_feats = reduce_mem_usage(df_valid_feats)
    else:
        df_train_feats = generate_all_features(df_train)
        print("Build Online Train Feats Finished.")

    df_train_feats = reduce_mem_usage(df_train_feats)


Build Train Feats Finished.
Build Valid Feats Finished.


In [9]:
df_valid_feats

,stock_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,...,bid_size_diff_10,dow,seconds,minute,global_median_size,global_std_size,global_ptp_size,global_median_price,global_std_price,global_ptp_price
5204892,0,0,3.753452e+06,-1,0.999875,11548975.0,NaN,NaN,0.999875,22940.000000,...,NaN,3,0,0,42514.429688,133007.187500,5.898990e+06,1.999694,0.003361,0.017414
5204893,1,0,9.859771e+05,-1,1.000245,3850034.0,NaN,NaN,0.999940,1967.900024,...,NaN,3,0,0,25425.230469,66472.632812,6.938986e+05,1.999824,0.005599,0.029370
5204894,2,0,5.991288e+05,1,1.000584,4359198.0,NaN,NaN,0.999918,4488.220215,...,NaN,3,0,0,26191.439453,75695.539062,1.069838e+06,2.000186,0.005342,0.051622
5204895,3,0,2.872318e+06,-1,0.999802,27129552.0,NaN,NaN,0.999705,16082.040039,...,NaN,3,0,0,41571.328125,93742.078125,1.928848e+06,1.999975,0.002911,0.018551
5204896,4,0,7.400591e+05,-1,0.999886,8880891.0,NaN,NaN,0.999720,19012.349609,...,NaN,3,0,0,33945.644531,80624.093750,1.604066e+06,1.999810,0.003727,0.017379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237887,195,540,2.440723e+06,-1,1.000317,28280362.0,0.999734,0.999734,1.000317,32257.039062,...,-10632.959961,0,0,9,51842.683594,98047.703125,2.761659e+06,1.999925,0.003057,0.014076
5237888,196,540,3.495105e+05,-1,1.000643,9187699.0,1.000129,1.000386,1.000643,205108.406250,...,88602.898438,0,0,9,42405.390625,77778.429688,4.596574e+05,2.000034,0.003425,0.017398
5237889,197,540,0.000000e+00,0,0.995789,12725436.0,0.995789,0.995789,0.995789,16790.660156,...,-20942.289062,0,0,9,30036.484375,71971.367188,1.575294e+06,1.999968,0.004696,0.020387
5237890,198,540,1.000899e+06,1,0.999210,94773272.0,0.999210,0.999210,0.998970,125631.718750,...,-129903.843750,0,0,9,304471.000000,354803.406250,2.159163e+06,1.999921,0.003148,0.015738


## Predict

In [10]:
import xgboost as xgb
import catboost as cb

lgb_path = ['ensemble_lgb/doblez_1.txt', 'ensemble_lgb/doblez_2.txt', 'ensemble_lgb/doblez_3.txt', 'ensemble_lgb/doblez_4.txt',
              'ensemble_lgb/doblez_5.txt', 'ensemble_lgb/doblez-conjunto.txt']

lgb_models = [lgb.Booster(model_file=model_filename) for model_filename in lgb_path]

xgb_path = ['ensemble_xgb/doblez_1.txt', 'ensemble_xgb/doblez_2.txt', 'ensemble_xgb/doblez_3.txt', 'ensemble_xgb/doblez_4.txt',
              'ensemble_xgb/doblez_5.txt', 'ensemble_xgb/doblez-conjunto.txt']

xgb_models = [xgb.Booster(model_file=model_filename) for model_filename in xgb_path]

cat_path = ['ensemble_cat/doblez_1.cbm', 'ensemble_cat/doblez_2.cbm', 'ensemble_cat/doblez_3.cbm', 'ensemble_cat/doblez_4.cbm',
              'ensemble_cat/doblez_5.cbm', 'ensemble_cat/doblez-conjunto.txt']

cat_models = [cb.CatBoostRegressor().load_model(model_filename) for model_filename in cat_path]

In [11]:
def zero_sum(prices, volumes):
    std_error = np.sqrt(volumes)
    step = np.sum(prices) / np.sum(std_error)
    out = prices - std_error * step
    return out


def predict(pred_models):
    y_min, y_max = -64, 64

    # Weights for each fold model
    model_weights = [1/len(pred_models)] * len(pred_models) 

    # Generate predictions for each model and calculate the weighted average
    predictions = np.zeros(len(df_valid))
    for model, weight in zip(pred_models, model_weights):
        predictions += weight * model.predict(df_valid_feats)

    predictions = zero_sum(predictions, df_valid['bid_size'] + df_valid['ask_size'])
    predictions = np.clip(predictions, y_min, y_max)

    return predictions

LightGBM and optimize ensemble weight

In [12]:
lgb_predictions = predict(lgb_models)
lgb_score = mean_absolute_error(lgb_predictions, df_valid['target'])

print(f'LightGBM score: {lgb_score}')

LightGBM score: 5.180033128821017


In [13]:
from scipy.optimize import minimize

def objective_function(weights):
    y_min, y_max = -64, 64

    # Weights for each fold model
    # model_weights = [1/len(pred_models)] * len(pred_models) 

    # Generate predictions for each model and calculate the weighted average
    predictions = np.zeros(len(df_valid))
    for model, weight in zip(lgb_models, weights):
        predictions += weight * model.predict(df_valid_feats)

    predictions = zero_sum(predictions, df_valid['bid_size'] + df_valid['ask_size'])
    predictions = np.clip(predictions, y_min, y_max)
    score = mean_absolute_error(predictions, df_valid['target'])
    return score


def find_weight():
    model_len = len(lgb_models)
    initial_weight = np.ones(model_len)/model_len
    bounds = [(0, 1)] * model_len
    result = minimize(objective_function, initial_weight, bounds=bounds, method='SLSQP')
    optimized_weights = result.x
    optimized_weights /= np.sum(optimized_weights)
    return optimized_weights

best_weight = find_weight()

print(f'Optimized weight: ', best_weight)

Optimized weight:  [1.26102329e-02 7.15351012e-01 2.72038755e-01 2.02381119e-17
 0.00000000e+00 4.58983371e-17]


In [14]:
y_min, y_max = -64, 64

# Generate predictions for each model and calculate the weighted average
predictions = np.zeros(len(df_valid))
for model, weight in zip(lgb_models, best_weight):
    predictions += weight * model.predict(df_valid_feats)

predictions = zero_sum(predictions, df_valid['bid_size'] + df_valid['ask_size'])
predictions = np.clip(predictions, y_min, y_max)
score = mean_absolute_error(predictions, df_valid['target'])

In [15]:
score

5.174559202103116